In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd
from pathlib import Path
import os

In [2]:
email = os.environ["USER_EMAIL"]
password = os.environ["USER_PASSWORD"]
URL = os.environ["URL"]

In [33]:
driver = webdriver.Chrome()

In [34]:
def login(url: str):
    driver.get(URL)
    time.sleep(10)
    WebDriverWait(driver=driver, timeout=10).until(
        lambda x: x.execute_script("return document.readyState === 'complete'"))
    driver.find_element("name", "email").send_keys(email)
    driver.find_element("name", "password").send_keys(password)
    driver.find_elements("css selector", "button")[1].click()

    WebDriverWait(driver=driver, timeout=10).until(
        lambda x: x.execute_script("return document.readyState === 'complete'"))

In [260]:
nav = driver.find_element("css selector", "div.mobile-navigator")

nav.find_elements("css selector", "a")[3].click()

In [139]:
def reload():
#     nav = driver.find_element("css selector", "div.mobile-navigator")
#     nav.find_elements("css selector", "a")[3].click()
    matches_container = driver.find_element("css selector", ".match-wrapper")
    matches = matches_container.find_elements("css selector", ".match-card")
    WebDriverWait(driver=driver, timeout=10).until(
        lambda x: x.execute_script("return document.readyState === 'complete'"))
    return matches

In [266]:
matches = reload()
len(matches)

10

In [40]:
def refresh():
    driver.back()
    driver.refresh()

In [84]:
def get_player_stats_url(match):
    
    teams = [team.text for team in match.find_elements("css selector", ".team")]
    opponent = [team for team in teams if team.lower() != "sporting lagos"][0]
    
    if match.find_element("css selector", "button").get_attribute("disabled") == "true":
            print("Button is not clickable")
            return
        
    try:
        match.find_element("css selector", "button").click()
    except:
        match.find_element("css selector", "button").click()
    time.sleep(3)
    
    try:
        driver.find_element("id", "TECHNICAL-ANALYSIS-tab").click()
    
    except:
        driver.find_element("id", "TECHNICAL-ANALYSIS-tab").click()
        
    time.sleep(5)
    
    player_names = [player_name.text for player_name in driver.find_elements("css selector", "a.player-name")]
    url = [player_name.get_attribute("href") for player_name in driver.find_elements("css selector", "a.player-name")]
    
    df = pd.DataFrame({"player_name": player_names, "url": url, "opponent": opponent})
    
    return df

In [50]:
players_dfs = []

In [141]:
df1 = get_player_stats_url(matches[5])

In [142]:
df1

,player_name,url,opponent
0,Christian Nwoke,https://manaja.app/app/players/pid-64457b1fd6d...,Joy Cometh
1,Ladeinde Femi,https://manaja.app/app/players/pid-64457a39d6d...,Joy Cometh
2,Patrick Egeonu,https://manaja.app/app/players/pid-64457a79d6d...,Joy Cometh
3,Daniel Segun,https://manaja.app/app/players/pid-64457aa8d6d...,Joy Cometh
4,Abiodun Balogun,https://manaja.app/app/players/pid-64457ae4d6d...,Joy Cometh
5,Odafi Emmanuel,https://manaja.app/app/players/pid-64457ba1d6d...,Joy Cometh
6,Rivio Ayemuenre,https://manaja.app/app/players/pid-64457964d6d...,Joy Cometh
7,Akanni Akeem,https://manaja.app/app/players/pid-64457be1d6d...,Joy Cometh
8,Chisom Orji,https://manaja.app/app/players/pid-64456d33d6d...,Joy Cometh
9,Oluwseun Taro,https://manaja.app/app/players/pid-644cf0a38e5...,Joy Cometh


In [143]:
players_dfs.append(df1)

In [144]:
refresh()

In [186]:
driver.get(players_dfs[0].url.values[0])

In [176]:
button = driver.find_element("css selector", "button.view-player-btn")
button.click()

In [180]:
title = driver.find_elements("css selector", ".stat-title")
val = driver.find_elements("css selector", ".stat-value")

In [185]:
print(title[-1].text)
print(val[-1].text)

Red
0


In [225]:
def get_additional_stat(url):
    driver.get(url)
    
    WebDriverWait(driver=driver, timeout=10).until(
        lambda x: x.execute_script("return document.readyState === 'complete'"))
    time.sleep(20)
    
    button = driver.find_element("css selector", "button.view-player-btn")
    try:
        button.click()
    except:
        button.click()
        
    title = [title.text for title in driver.find_elements("css selector", ".stat-title")]
    val = [val.text for val in driver.find_elements("css selector", ".stat-value")]
    
    df = pd.DataFrame()
    
    for i in range(len(title)):
        df[title[i]] = [val[i]]
        
    return df

In [234]:
def add_stats_to_df(df):
    df_list = []
    urls = df.url.values
    
    for url in urls:
        df = get_additional_stat(url)
        
        df_list.append(df)
        
    return pd.concat(df_list).reset_index(drop=True)
    

In [256]:
for index, df in enumerate(players_dfs[6:]):
    print(f"getting data for df at index {index + 6}")
    add = add_stats_to_df(df)
    df = pd.concat([df, add], axis=1)
    players_dfs[index + 6] = df


getting data for df at index 6
getting data for df at index 7
getting data for df at index 8
getting data for df at index 9
getting data for df at index 10
getting data for df at index 11


In [232]:
players_dfs[0] = pd.concat([players_dfs[0], add.reset_index(drop=True)], axis=1)

In [367]:
players_dfs[11]

,player_name,url,opponent,Minutes played,Starting Lineup,Substituted Out,Goals,Assist,Shots,Free kick,...,Saves,Catches,Goal conceded,Offside,Goalkeeper Drop,Goalkeeper Throw,Possession Lost,Punches,Yellow,Red
0,Christian Nwoke,https://manaja.app/app/players/pid-64457b1fd6d...,Joy Cometh,93,Yes,No,0,0,0,0,...,0,1,2,0,0,0,0,0,0,0
1,Ladeinde Femi,https://manaja.app/app/players/pid-64457a39d6d...,Joy Cometh,93,Yes,No,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Patrick Egeonu,https://manaja.app/app/players/pid-64457a79d6d...,Joy Cometh,93,Yes,No,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,Daniel Segun,https://manaja.app/app/players/pid-64457aa8d6d...,Joy Cometh,93,Yes,No,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Abiodun Balogun,https://manaja.app/app/players/pid-64457ae4d6d...,Joy Cometh,93,Yes,No,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Odafi Emmanuel,https://manaja.app/app/players/pid-64457ba1d6d...,Joy Cometh,93,Yes,No,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
6,Rivio Ayemuenre,https://manaja.app/app/players/pid-64457964d6d...,Joy Cometh,99,Yes,Yes,0,0,2,1,...,0,0,0,0,0,0,0,0,0,0
7,Akanni Akeem,https://manaja.app/app/players/pid-64457be1d6d...,Joy Cometh,75,Yes,Yes,0,0,1,0,...,0,0,0,0,0,0,3,0,0,0
8,Chisom Orji,https://manaja.app/app/players/pid-64456d33d6d...,Joy Cometh,93,Yes,No,0,0,1,1,...,0,0,0,0,0,0,3,0,0,0
9,Oluwseun Taro,https://manaja.app/app/players/pid-644cf0a38e5...,Joy Cometh,87,Yes,Yes,0,0,2,0,...,0,0,0,0,0,0,5,0,0,0


In [368]:
players_map = {
    "Asibe Nathaniel": "GK",
    "Christian Nwoke": "GK",
    "Ekoi Agbor": "GK",
    "Abiodun Ozaveche": "DF",
    "Abiodun Balogun": "DF",
    "Annor Isaac": "DF",
    "Nnamdi Anthony": "DF",
    "Anthony Nnamdi": "DF",
    "Chiemeka Nwokeji": "DF",
    "Daniel Segun": "DF",
    "Ekene OLisema": "DF",
    "Ladeinde Femi": "DF",
    "Nelson Onyeka" : "DF",
    "Patrick Egeonu": "DF",
    "Salawudeen Aliu": "DF",
    "Abdulrazaq Shaibu": "MF",
    "Akanni Akeem": "MF",
    "Chinedu Ufere": "MF",
    "Chukwuemeka Kalu": "MF",
    "Dauebimone Mehra": "MF",
    "Ndedi Henry": "MF",
    "Odafi Emmanuel": "MF",
    "Rivio Ayemuenre": "MF",
    "Temidayo Okonlawon": "MF",
    "Vincent Akpikpi": "MF",
    "Akpesiri Naibe": "FW",
    "Bartholomew Daniel": "FW",
    "Chisom Orji": "FW",
    "Clement Naantaum": "FW",
    "Godwin Odibo": "FW",
    "Jonathan Alukwu": "FW",
    "Junior Lokosa": "FW",
    "Kosoko Toheeb": "FW",
    "Mamah Osan": "FW",
    "Odunayo Abiodun": "FW",
    "Odunayo Ayo": "FW",
    "Oluwseun Taro": "FW",
    "Saturday Okon": "FW",
    "Udoh Godwin": "FW",
    "Williams Obande": "FW",
    "Wisdom Samuel": "FW" 
}

In [371]:
player_df = pd.concat(players_dfs).reset_index(drop=True)

In [377]:
columns = player_df.columns.str.replace("url", "player_position")

In [379]:
player_df = player_df.assign(player_position = player_df.player_name.map(players_map)).drop("url", axis=1)[columns]

In [381]:
player_df.to_csv("sheets/player_data.csv", index=False)

In [340]:
driver.close()